In [7]:
from pymongo import MongoClient
import pandas as pd
import spacy

# # Connect to MongoDB database
# client = MongoClient('mongodb://localhost:27017/')
# db = client['pidb']
# collection = db['emails_bodies']

# # Retrieve data from MongoDB collection
# data = list(collection.find({}, {"_id": 0}))

# # Convert data to a Pandas DataFrame
# df = pd.DataFrame(data)


# # Close MongoDB connection
# client.close()


df = pd.read_csv('../datasets/nazario_trec07_text.csv')


In [8]:
# Set the display option to show the full content of all columns
pd.set_option('display.max_colwidth', None)
df

body  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

### 1. Remove any HTML tags and punctuation
This step is important because HTML tags and special characters can interfere with the natural language processing algorithms and can add noise to the dataset. 

In [9]:
from bs4 import BeautifulSoup
import re



# define lambda functions for each step
remove_html = lambda body: BeautifulSoup(str(body), 'html.parser').get_text()
remove_punct = lambda body: re.sub(r'[^\w\s]', '', body)
remove_js = lambda body: re.sub(r'<script.*?>.*?</script>', '', body, flags=re.DOTALL)

# apply lambda functions to the 'body' column
df['body'] = df['body'].apply(remove_html)
df['body'] = df['body'].apply(remove_punct)
df['body'] = df['body'].apply(remove_js)

df


/tmp/ipykernel_3717/801742098.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  remove_html = lambda body: BeautifulSoup(str(body), 'html.parser').get_text()
/home/user01/miniconda3/envs/pi/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


,body,label
0,bThis text is part of the internal format of your mail folder and is notna real message It is created automatically by the mail system softwarenIf deleted important folder data will be lost and it will be recreatednwith the data reset to initial valuesn,1
1,nan,1
2,nan,1
3,nan,1
4,nan,1
...,...,...
32162,bnnnCNN Alerts My Custom Alertnnnnnnnnnnn n n nnnAlert Name My Custom AlertnAustralia bans Snoop Doggnn042607 0429 AM EDTnUS rapper Snoop Dogg has been barred from entering Australia after failing a character test Australias government said on ThursdaynFULL STORYnnnnnYou have agreed to receive this email from CNNcom as a result of your CNNcom preference settingsnTo manage your settings click here nTo alter your alter your alert criteria or frequency or to unsubscribe from receiving custom email alerts click herenRefer a friend or colleague to CNNs FREE personalized alerting servicennnCable News Network LP LLLP One CNN Center Atlanta Georgia 30303n 2007 Cable News Network LP LLLPnA Time Warner Company All Rights ReservednTerms under which this service is provided to younRead our privacy guidelines Contact usnn n n nnnnn,0
32163,bHello sm1liesnnDesign the look and feel of the 2007 TCO program This contest begins at 200 PM Eastern Daylight Time UTCGMT 4 on Tuesday April 10 2007 The submission deadline for this contest is 200 PM Eastern Daylight Time UTCGMT 4 on Tuesday April 24 2007nnThe contestant with the winning submission will receive 2000 This is a TCOeligible contestnnFor all of the contest details check out httpstudiotopcodercommoduleViewActiveContestsn nBest of luck to you at the Studio nnn The TopCoder Studio Team nnnnYou have received this email because you indicated in your TopCoder profile that you want to be notified of TopCoder Studio andor Graphic Design competitions If you no longer wish to receive these emails log in to TopCoder or TopCoder Studio and click on Update My Profile In the Email Notification section of your profile you have the option to enable or disable receipt of various types of emails Be sure to save any changes to your member profilennnhttpstudiotopcodercomnTopCoder Incn703 Hebron AvenuenGlastonbury CT 06033nPhone 18668672633nnn,0
32164,b189253281177738941336JavaMailjbossweb22brainbenchcomnContentType multipartalternative boundary177402271177738941339JavaMailjbossweb22brainbenchcomnn177402271177738941339JavaMailjbossweb22brainbenchcomnContentType textplain charsetusasciinContentTransferEncoding 7bitnnYou are invited to participate in TechSaycoms IT research panelnn nnDear Zombienn nnWhen you registered with Brainbench you also expressed interest in learning more about one of our membership benefits whereby you get paid cash in return for completing online surveys that shape the future of technologynn nnOur partner in this venture is techsaycom a global technology research firm This riskfree no obligation opportunity allows you to join the thousands of other Brainbench IT professionals who are making money while influencing the future of technology nn nnThe process is simplenn nn1 You will be taking online surveys directly related to your profession and providing feedback that drives the direction of IT research and product development Receive emailed survey invitations from surveystechsaycom nn nn2 For each survey you complete you will be awarded a minimum of 200 TekPoints nn nn3 Once you have attained a minimum of 1000 TekPoints redeem them for a 5000 check in your local currencynn nnJoin TechSaycom today atnnhttpwwwtechsaycomdefaultphppageactivate_bbphppuidc20df62ebd5e20ff9d92bfd59aa7bfacnn nnIf this opportunity does not interest you and you would like to unsubscribe from future TechSaycom mailings email unsubscribe to surveystechsaycomnn nnTechSaycom 2835 82nd Ave SE Suite S100 Mercer Island WA 98040nn nnn177402271177738941339JavaMailjbossweb22brainbenchcomnContentType texthtmlnContentTransferEncoding 7bitnnnnn nn nnnnnYou are invited to participate in TechSaycoms IT research pa

### 2. Convert text to lowercase
This step is performed to ensure consistency for further processing

In [10]:
lowercase = lambda text: text.lower()

df['body'] = df['body'].apply(lowercase)

df

,body,label
0,bthis text is part of the internal format of your mail folder and is notna real message it is created automatically by the mail system softwarenif deleted important folder data will be lost and it will be recreatednwith the data reset to initial valuesn,1
1,nan,1
2,nan,1
3,nan,1
4,nan,1
...,...,...
32162,bnnncnn alerts my custom alertnnnnnnnnnnn n n nnnalert name my custom alertnaustralia bans snoop doggnn042607 0429 am edtnus rapper snoop dogg has been barred from entering australia after failing a character test australias government said on thursdaynfull storynnnnnyou have agreed to receive this email from cnncom as a result of your cnncom preference settingsnto manage your settings click here nto alter your alter your alert criteria or frequency or to unsubscribe from receiving custom email alerts click herenrefer a friend or colleague to cnns free personalized alerting servicennncable news network lp lllp one cnn center atlanta georgia 30303n 2007 cable news network lp lllpna time warner company all rights reservednterms under which this service is provided to younread our privacy guidelines contact usnn n n nnnnn,0
32163,bhello sm1liesnndesign the look and feel of the 2007 tco program this contest begins at 200 pm eastern daylight time utcgmt 4 on tuesday april 10 2007 the submission deadline for this contest is 200 pm eastern daylight time utcgmt 4 on tuesday april 24 2007nnthe contestant with the winning submission will receive 2000 this is a tcoeligible contestnnfor all of the contest details check out httpstudiotopcodercommoduleviewactivecontestsn nbest of luck to you at the studio nnn the topcoder studio team nnnnyou have received this email because you indicated in your topcoder profile that you want to be notified of topcoder studio andor graphic design competitions if you no longer wish to receive these emails log in to topcoder or topcoder studio and click on update my profile in the email notification section of your profile you have the option to enable or disable receipt of various types of emails be sure to save any changes to your member profilennnhttpstudiotopcodercomntopcoder incn703 hebron avenuenglastonbury ct 06033nphone 18668672633nnn,0
32164,b189253281177738941336javamailjbossweb22brainbenchcomncontenttype multipartalternative boundary177402271177738941339javamailjbossweb22brainbenchcomnn177402271177738941339javamailjbossweb22brainbenchcomncontenttype textplain charsetusasciincontenttransferencoding 7bitnnyou are invited to participate in techsaycoms it research panelnn nndear zombienn nnwhen you registered with brainbench you also expressed interest in learning more about one of our membership benefits whereby you get paid cash in return for completing online surveys that shape the future of technologynn nnour partner in this venture is techsaycom a global technology research firm this riskfree no obligation opportunity allows you to join the thousands of other brainbench it professionals who are making money while influencing the future of technology nn nnthe process is simplenn nn1 you will be taking online surveys directly related to your profession and providing feedback that drives the direction of it research and product development receive emailed survey invitations from surveystechsaycom nn nn2 for each survey you complete you will be awarded a minimum of 200 tekpoints nn nn3 once you have attained a minimum of 1000 tekpoints redeem them for a 5000 check in your local currencynn nnjoin techsaycom today atnnhttpwwwtechsaycomdefaultphppageactivate_bbphppuidc20df62ebd5e20ff9d92bfd59aa7bfacnn nnif this opportunity does not interest you and you would like to unsubscribe from future techsaycom mailings email unsubscribe to surveystechsaycomnn nntechsaycom 2835 82nd ave se suite s100 mercer island wa 98040nn nnn177402271177738941339javamailjbossweb22brainbenchcomncontenttype texthtmlncontenttransferencoding 7bitnnnnn nn nnnnnyou are invited to participate in techsaycoms it research pa

### 3. Remove stop words such as "and", "the", "a", etc. 
These words typically do not provide much meaning to the email and can be removed without affecting the overall context

In [11]:

# load the Spacy English model
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
nlp.max_length=5862775

def remove_stopwords(text):
    doc = nlp(text)
    return ' '.join([token.text for token in doc if not token.is_stop])

df['body'] = df['body'].apply(remove_stopwords)



In [12]:
df.to_csv('../datasets/preprocessed_nazario_trec07_text.csv', index=False)


### 4. Convert the text into numerical features using techniques such as bag-of-words, TF-IDF

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer


# define a custom tokenizer for Spacy
def spacy_tokenizer(doc):
    tokens = nlp(doc)
    return [token.lemma_ for token in tokens]

# create a TfidfVectorizer object with the Spacy tokenizer
vectorizer = TfidfVectorizer(tokenizer=spacy_tokenizer)

# fit and transform the 'emailbody' column of your dataframe
X = vectorizer.fit_transform(df['body'])

# print the transformed feature matrix
print(X.toarray())


/home/user01/miniconda3/envs/pi/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


KeyboardInterrupt: 